In [1]:
%cd ../

/mnt/kireev/pycharm-deploy/vtb


In [2]:
import pickle
import random

In [3]:
from glob import glob

In [4]:
import numpy as np
import pandas as pd

In [5]:
import torch
import pytorch_lightning as pl

In [6]:
from pyhocon import ConfigFactory

In [7]:
import matplotlib.pyplot as plt

In [8]:
from dltranz.data_load.iterable_processing.category_size_clip import CategorySizeClip
from dltranz.data_load import augmentation_chain
from dltranz.data_load.augmentations.seq_len_limit import SeqLenLimit
from dltranz.data_load.augmentations.random_slice import RandomSlice

from dltranz.seq_encoder import create_encoder

from dltranz.metric_learn.sampling_strategies import get_sampling_strategy
from dltranz.metric_learn.losses import get_loss

from dltranz.tb_interface import get_scalars

from dltranz.data_preprocessing.pandas_preprocessor import PandasDataPreprocessor

In [9]:
from vtb_code.data import PairedDataset, paired_collate_fn, DropDuplicate
from vtb_code.metrics import PrecisionK, MeanReciprocalRankK, ValidationCallback

from vtb_code.preprocessing import trx_types, click_types, trx_to_torch, click_to_torch

In [10]:
FOLD_ID = 0

In [11]:
fold_id_test = FOLD_ID

In [12]:
folds_count = len(glob('data/train_matching_*.csv'))
folds_count

6

In [13]:
# fold_id_valid = np.random.choice([i for i in range(folds_count) if i != fold_id_test], size=1)[0]
fold_id_valid = (fold_id_test + 1) % folds_count
fold_id_valid

1

In [14]:
df_matching_train = pd.concat([pd.read_csv(f'data/train_matching_{i}.csv')
                              for i in range(folds_count) 
                              if i not in (fold_id_test, fold_id_valid)])
df_matching_valid = pd.read_csv(f'data/train_matching_{fold_id_valid}.csv')
df_matching_test = pd.read_csv(f'data/train_matching_{fold_id_test}.csv')

In [15]:
[len(df) for df in [df_matching_train, df_matching_valid, df_matching_test]]

[11720, 2930, 2931]

In [17]:
df_trx_train = pd.concat([trx_types(pd.read_csv(f'data/transactions_{i}.csv'))
                              for i in range(folds_count) 
                              if i not in (fold_id_test, fold_id_valid)])
df_trx_valid = trx_types(pd.read_csv(f'data/transactions_{fold_id_valid}.csv'))
df_trx_test = trx_types(pd.read_csv(f'data/transactions_{fold_id_test}.csv'))
df_trx_puzzle = trx_types(pd.read_csv(f'data/transactions_unmatched.csv'))

In [19]:
df_click_train = pd.concat([click_types(pd.read_csv(f'data/clickstream_{i}.csv'))
                              for i in range(folds_count) 
                              if i not in (fold_id_test, fold_id_valid)])
df_click_valid = click_types(pd.read_csv(f'data/clickstream_{fold_id_valid}.csv'))
df_click_test = click_types(pd.read_csv(f'data/clickstream_{fold_id_test}.csv'))
df_click_puzzle = click_types(pd.read_csv(f'data/clickstream_unmatched.csv'))

In [21]:
preprocessor_trx = PandasDataPreprocessor(
    col_id='user_id',
    cols_event_time='event_time',
    time_transformation='none',
    cols_category=["mcc_code", "currency_rk"],
    cols_log_norm=["transaction_amt"],
    cols_identity=[],
    print_dataset_info=False,
)

preprocessor_click = PandasDataPreprocessor(
    col_id='user_id',
    cols_event_time='event_time',
    time_transformation='none',
    cols_category=['cat_id', 'level_0', 'level_1', 'level_2'],
    cols_log_norm=[],
    cols_identity=['new_uid'],
    print_dataset_info=False,
)

In [24]:
features_trx_train = dict(trx_to_torch(preprocessor_trx.fit_transform(df_trx_train)))
features_trx_valid = dict(trx_to_torch(preprocessor_trx.transform(df_trx_valid)))
features_trx_test = dict(trx_to_torch(preprocessor_trx.transform(df_trx_test)))
features_trx_puzzle = dict(trx_to_torch(preprocessor_trx.transform(df_trx_puzzle)))

In [25]:
features_click_train = dict(click_to_torch(preprocessor_click.fit_transform(df_click_train)))
features_click_valid = dict(click_to_torch(preprocessor_click.transform(df_click_valid)))
features_click_test = dict(click_to_torch(preprocessor_click.transform(df_click_test)))
features_click_puzzle = dict(click_to_torch(preprocessor_click.transform(df_click_puzzle)))

In [26]:
with open(f'data/features_f{FOLD_ID}.pickle', 'wb') as f:
    pickle.dump((
        features_trx_train,
        features_trx_valid,
        features_trx_test,
        features_trx_puzzle,
        features_click_train,
        features_click_valid,
        features_click_test,
        features_click_puzzle,
    ), f)

In [27]:
%%time
with open(f'data/features_f{FOLD_ID}.pickle', 'rb') as f:
    (
        features_trx_train,
        features_trx_valid,
        features_trx_test,
        features_trx_puzzle,
        features_click_train,
        features_click_valid,
        features_click_test,
        features_click_puzzle,
    ) = pickle.load(f)

CPU times: user 14.8 s, sys: 3.93 s, total: 18.7 s
Wall time: 18.8 s
